In [1]:
!pip install pandas
!pip install matplotlib
!pip install folium
!pip install geopy

In [2]:
import pandas as pd
data = pd.read_csv('cheongju_restaurant_data.csv', encoding='cp949')
print(data.head())

     업종명              업소명                                           소재지(도로명)  \
0  일반음식점           랑성유황오리                                청주시 상당구 낭성면 관정1길 10   
1  일반음식점  비에이치씨(BHC)청주용정점                     청주시 상당구 영운천로133번길 33  1층 (금천동)   
2  일반음식점         BHC방서동남점                청주시 상당구 월평로238번길 3-5  1층 101호 (용암동)   
3  일반음식점           bhc성안점  청주시 상당구 용담로 7  주5동 1층 129호  130호 (문화동  문화동 센트럴...   
4  일반음식점      투썸플레이스청주동남점                   청주시 상당구 중고개로125번길 19  1~4층 (용암동)   

                       소재지(지번)         소재지전화    업태명  
0      청주시 상당구 낭성면 관정리 413-10   043-285-6767     기타  
1          청주시 상당구 금천동 452 1층            NaN  호프/통닭  
2            청주시 상당구 용암동 2035   043-297-1577  호프/통닭  
3  청주시 상당구 문화동 118 문화동 센트럴칸타빌            NaN  호프/통닭  
4      청주시 상당구 용암동 189-3 1~4층   043-298-1200    경양식  


In [4]:
import folium
g_map = folium.Map(location=[36.6286, 127.4577], zoom_start=15)
g_map

In [6]:
address = data['소재지(도로명)']
print(type(address[0]))
print(address.head())

<class 'str'>
0                                  청주시 상당구 낭성면 관정1길 10
1                       청주시 상당구 영운천로133번길 33  1층 (금천동)
2                  청주시 상당구 월평로238번길 3-5  1층 101호 (용암동)
3    청주시 상당구 용담로 7  주5동 1층 129호  130호 (문화동  문화동 센트럴...
4                     청주시 상당구 중고개로125번길 19  1~4층 (용암동)
Name: 소재지(도로명), dtype: object


In [7]:
for i in range(len(address)):
    a = str(address[i]).split(' ')
    address[i] = ' '.join(a[0:4])
print(address.head())

0        청주시 상당구 낭성면 관정1길
1    청주시 상당구 영운천로133번길 33
2    청주시 상당구 월평로238번길 3-5
3           청주시 상당구 용담로 7
4    청주시 상당구 중고개로125번길 19
Name: 소재지(도로명), dtype: object


In [8]:
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

def geocoding(address):
    geo = geo_local.geocode(address)
    if geo is None: return [0, 0]
    x_y = [geo.latitude, geo.longitude]
    return x_y

In [10]:
latitude = []
longitude = []

for i in range(100):
    a = geocoding(address[i])
    latitude.append(a[0])
    longitude.append(a[1])

data = data[:100]
res_data = pd.DataFrame({'업종명': data['업종명'], '업소명': data['업소명'], '소재지(도로명)': data['소재지(도로명)'],
                        '소재지(지번)': data['소재지(지번)'], '소재지전화': data['소재지전화'], '업태명': data['업태명'],
                        '위도': latitude, '경도': longitude})